In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from src.models import ResNet, MNIST_CNN, CIFAR_CNN
from src.helpers import evaluate_rob_accuracy, evaluate_clean_accuracy, load_model, safe_model,_evaluate_model
from src.data_loader import load_torchvision_dataset, load_imagenette
#from src.pruning import identify_layers, _evaluate_sparsity

#device = torch.device("cuda:0")
device = torch.device("cpu")

dtype = torch.float32

# Initialization

In [2]:
#model = ResNet()
#model = MNIST_CNN()
model = CIFAR_CNN()


identifying layers


In [3]:
train_loader, test_loader = load_torchvision_dataset('CIFAR10')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
PATH = './saved-models/CIFAR-baseline-150-epochs.pth'
model = load_model(model, PATH)

# before

# Testing

In [8]:
model.prune_magnitude_global_struct(.3, structure='kernel')

({'c1.mask': 0.02083333333333337,
  'c2.mask': 0.18359375,
  'c3.mask': 0.27734375,
  'c4.mask': 0.353515625,
  'fc2.mask': 0.0,
  'fc3.mask': 0.0},
 0.04131736526946106)

In [9]:
evaluate_clean_accuracy(model, test_loader, device)

TypeError: tensor should be a torch tensor. Got <class 'PIL.Image.Image'>.

In [ ]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

In [ ]:
model = load_model(model, PATH)

In [ ]:
model.prune_magnitude_global_struct(.3, structure='filter')

In [ ]:
evaluate_clean_accuracy(model, test_loader, device)

In [ ]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

In [ ]:
model = load_model(model, PATH)

In [ ]:
model.prune_magnitude_global_unstruct(.3)

In [ ]:
evaluate_clean_accuracy(model, test_loader, device)

In [ ]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

In [ ]:
model = load_model(model, PATH)

In [ ]:
model.prune_magnitude_local_struct(.3, structure='kernel')

In [ ]:
evaluate_clean_accuracy(model, test_loader, device)

In [ ]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

In [ ]:
model = load_model(model, PATH)

In [ ]:
model.prune_magnitude_local_struct(.3, structure='filter')

In [ ]:
evaluate_clean_accuracy(model, test_loader, device)

In [ ]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

In [ ]:
model = load_model(model, PATH)

In [ ]:
model.prune_magnitude_local_unstruct(.3, scope='layer')

In [ ]:
evaluate_clean_accuracy(model, test_loader, device)

In [ ]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

In [ ]:
model = load_model(model, PATH)

In [ ]:
model.prune_magnitude_local_unstruct(.3, scope='filter')

In [ ]:
evaluate_clean_accuracy(model, test_loader, device)

In [ ]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

In [ ]:
model = load_model(model, PATH)

In [ ]:
model.prune_random_local_struct(.3, structure='kernel')

In [ ]:
evaluate_clean_accuracy(model, test_loader, device)

In [ ]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

In [ ]:
model = load_model(model, PATH)

In [ ]:
model.prune_random_local_struct(.3, structure='filter')

In [ ]:
evaluate_clean_accuracy(model, test_loader, device)

In [ ]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

In [ ]:
model = load_model(model, PATH)

In [ ]:
model.prune_random_local_unstruct(.3)

In [ ]:
evaluate_clean_accuracy(model, test_loader, device)

In [ ]:
evaluate_rob_accuracy(model, test_loader, device, epsilon=8/255, attack='FGSM')

In [ ]:
model = load_model(model, PATH)

In [ ]:
_prune_magnitude_local_unstruct(model, .6, scope='layer')

In [ ]:
evaluate_clean_accuracy(model, test_loader, device)

In [ ]:
def prune_filters(model, ratio, weights):
        all_filters = []
        all_masks = []
        vals = []
        for i, layer_to_prune in enumerate(conv_weights):
            # convert from e.g. (3,3,64,128) to (128,64,3,3)
            oihw_weights = helpers.convert_from_hwio_to_oihw(weights[layer_to_prune])
            oihw_mask = helpers.convert_from_hwio_to_oihw(weights[model.conv_masks[i]])
            no_of_filters = oihw_weights.shape[0]
            
            #calculate average magnitude for each filter
            vals = vals + [np.sum(np.abs(single_filter)) / np.prod(single_filter.shape) for single_filter in oihw_weights]
            all_filters = list(all_filters) +  list(oihw_weights)
            all_masks = list(all_masks) + list(oihw_mask)
        no_of_filters_to_prune = int(np.round(ratio * len(vals)))
        filters_to_prune = np.argsort(vals)[:no_of_filters_to_prune]
        
        for filter_to_prune in filters_to_prune:
            all_filters[filter_to_prune] = tf.zeros(all_filters[filter_to_prune].shape) 
            all_masks[filter_to_prune] = tf.zeros(all_filters[filter_to_prune].shape) 
        z = 0
        for i, _ in enumerate(conv_weights):
            original_shape = helpers.convert_from_hwio_to_oihw(weights[layer_to_prune]).shape
            pruned_layer = tf.reshape(all_filters[z:z + original_shape[0]], original_shape)
            pruned_mask = tf.reshape(all_masks[z:z + original_shape[0]], original_shape)
            weights[layer_to_prune] = helpers.convert_from_oihw_to_hwio(pruned_layer)
            weights[model.conv_masks[i]] = helpers.convert_from_oihw_to_hwio(pruned_mask)
            z = z + original_shape[0]

In [ ]:
for p in model.parameters():
    print(p)

In [ ]:
conv_weights, conv_masks, fully_connected_weights, fully_connected_masks = find_layers(model)

In [ ]:

ratio = .1
params = model.state_dict()
for i, _ in enumerate(conv_weights):
    weights = params[conv_weights[i]]
    mask = params[conv_masks[i]]
    shape = weights.shape
    flat_weights = weights.flatten()
    flat_mask = mask.flatten()
    
    no_of_weights_to_prune = int(round(ratio * len(flat_weights)))
    non_zero_weights = torch.nonzero(flat_weights)
    no_of_weights_to_prune_left = int(no_of_weights_to_prune - (len(flat_weights) - len(non_zero_weights)) )
    non_zero_weights = non_zero_weights[torch.randperm(non_zero_weights.nelement())] #shuffle tensor
    indices_to_delete = non_zero_weights[:no_of_weights_to_prune_left]
    
    for idx_to_delete in indices_to_delete:
        flat_mask[idx_to_delete] = 0
        flat_weights[idx_to_delete] = 0

    params[conv_weights[i]] = flat_weights.view(shape)
    params[conv_masks[i]] = flat_mask.view(shape)
model.load_state_dict(params)

In [ ]:

ratio = .1
params = model.state_dict()
for i, _ in enumerate(fully_connected_weights):
    weights = params[fully_connected_weights[i]]
    mask = params[fully_connected_masks[i]]
    shape = weights.shape
    flat_weights = weights.flatten()
    flat_mask = mask.flatten()
    
    no_of_weights_to_prune = int(round(ratio * len(flat_weights)))
    non_zero_weights = torch.nonzero(flat_weights)
    no_of_weights_to_prune_left = int(no_of_weights_to_prune - (len(flat_weights) - len(non_zero_weights)) )
    non_zero_weights = non_zero_weights[torch.randperm(non_zero_weights.nelement())] #shuffle tensor
    indices_to_delete = non_zero_weights[:no_of_weights_to_prune_left]
    
    for idx_to_delete in indices_to_delete:
        flat_mask[idx_to_delete] = 0
        flat_weights[idx_to_delete] = 0

    params[fully_connected_weights[i]] = flat_weights.view(shape)
    params[fully_connected_masks[i]] = flat_mask.view(shape)
model.load_state_dict(params)

In [ ]:
for key in model.state_dict().keys():
    p = model.state_dict()[key]
    total = p.nelement()
    non_zero = len(torch.nonzero(p.view(-1)))
    print(total)
    print(non_zero)
    print('ratio: ', non_zero/total)